In [2]:
from pyspark.sql import SparkSession

In [3]:
# Khởi tạo SparkSession
spark = SparkSession \
    .builder \
    .appName("Import to warehouse") \
    .config("spark.master", "spark://192.168.1.105:7077") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/16 14:53:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
# Đọc dữ liệu từ datalake 
df = spark.read.parquet("hdfs://192.168.1.105:9000/datalake/data1612")

In [14]:
# Hiển thị dữ liệu để xác nhận
df.show()

+---+---+------+--------------+--------------------+-----------------+------+---------+----------------+
| id|Age|Gender|EducationLevel|            JobTitle|YearsofExperience|Salary|  Country|            Race|
+---+---+------+--------------+--------------------+-----------------+------+---------+----------------+
|  0| 32|  Male|    Bachelor's|   Software Engineer|              5.0| 90000|       UK|           White|
|  1| 28|Female|      Master's|        Data Analyst|              3.0| 65000|      USA|        Hispanic|
|  2| 45|  Male|           PhD|      Senior Manager|             15.0|150000|   Canada|           White|
|  3| 36|Female|    Bachelor's|     Sales Associate|              7.0| 60000|      USA|        Hispanic|
|  4| 52|  Male|      Master's|            Director|             20.0|200000|      USA|           Asian|
|  5| 29|  Male|    Bachelor's|   Marketing Analyst|              2.0| 55000|      USA|        Hispanic|
|  6| 42|Female|      Master's|     Product Manager|   

In [15]:
df=df.dropna()

In [16]:
df.drop("id")

DataFrame[Age: int, Gender: string, EducationLevel: string, JobTitle: string, YearsofExperience: double, Salary: int, Country: string, Race: string]

In [17]:
df.write.parquet("hdfs://192.168.1.105:9000/datawarehouse/spark/data1612", mode="overwrite")

23/12/16 15:35:03 ERROR TaskSchedulerImpl: Lost executor 1 on 192.168.1.100: Worker shutting down
23/12/16 15:35:35 ERROR TaskSchedulerImpl: Lost executor 2 on 192.168.1.106: Command exited with code 129
23/12/16 15:35:54 ERROR TaskSchedulerImpl: Lost executor 5 on 192.168.1.106: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


In [10]:
replacement_rules = [
    (df["EducationLevel"] == "Bachelor's Degree", "Bachelor's"),
    (df["EducationLevel"] == "Master's Degree", "Master's"),
    (df["EducationLevel"] == "phD", "PhD")
]


In [12]:
from pyspark.sql.functions import when
for condition, value in replacement_rules:
    df = df.withColumn("EducationLevel", when(condition, value).otherwise(df["EducationLevel"]))

AnalysisException: [MISSING_ATTRIBUTES.RESOLVED_ATTRIBUTE_APPEAR_IN_OPERATION] Resolved attribute(s) "EducationLevel" missing from "id", "Age", "Gender", "EducationLevel", "JobTitle", "YearsofExperience", "Salary", "Country", "Race" in operator !Project [id#0, Age#1, Gender#2, CASE WHEN (EducationLevel#3 = Master's Degree) THEN Master's ELSE EducationLevel#64 END AS EducationLevel#74, JobTitle#4, YearsofExperience#5, Salary#6, Country#7, Race#8]. Attribute(s) with the same name appear in the operation: "EducationLevel".
Please check if the right attribute(s) are used.;
!Project [id#0, Age#1, Gender#2, CASE WHEN (EducationLevel#3 = Master's Degree) THEN Master's ELSE EducationLevel#64 END AS EducationLevel#74, JobTitle#4, YearsofExperience#5, Salary#6, Country#7, Race#8]
+- Project [id#0, Age#1, Gender#2, CASE WHEN (EducationLevel#3 = Bachelor's Degree) THEN Bachelor's ELSE EducationLevel#3 END AS EducationLevel#64, JobTitle#4, YearsofExperience#5, Salary#6, Country#7, Race#8]
   +- Rel